# i.MX 93 AHAB with flashloader 

This notebook describes how to build the bootable AHAB image with ELE firmware and flashloader firmware with MBOOT protocol support and how to use NXPELE and BLHOST apps.

## Set up the environment

- SPSDK is needed with examples extension. pip install spsdk[examples] (refer to the installation documentation)
- This demo was tested with i.MX93 EVK board.

## Image preparation
First we need to prepare AHAB image for CM33 core containing ELE firmware and our flashloader. We can use SPSDK *nxpimage* tool for image preparation.
Configuration file that generates signed AHAB image containing ELE FW and flashloader might look like this:

```yaml
---
# ----------------------------------------------------------------------------------------------------
# ===========  Advanced High-Assurance Boot Configuration template for rt1180.  ===========
# ----------------------------------------------------------------------------------------------------
#                                        == General Options ==
family: mx93 # [Required], MCU family, Family identifier including the chip revision. If revision is not present, latest revision is used as default., Possible options:['rt118x']
revision: a0 # [Optional], MCU revision, Revision of silicon, Possible options:['a0']
image_type: serial_downloader # [Required], Type of image, The final use of image, this setting is changing the style of offsets in final container., Possible options:['xip', 'non_xip', 'serial_downloader']
output: ..\outputs/flashloader_signed.bin # [Required], Output AHAB file name, Revision of silicon
containers: # [Required], List of containers present in AHAB., The order of containers in the list defines the order in AHAB.
  - # ----------------------------------------------------------------------------------------------------
    #                     == Binary Container format to add to AHAB image ==
    # ----------------------------------------------------------------------------------------------------
    binary_container: # [Required], Binary AHAB container
      path: inputs/mx93a0-ele-fw.img # [Required], The AHAB container binary file, The binary file that contains AHAB "my_binary_container.bin
  - # [Example of possible configuration #1]
    # ----------------------------------------------------------------------------------------------------
    #                  == Configuration Container format to add to AHAB image ==
    # ----------------------------------------------------------------------------------------------------
    container: # [Required], AHAB Container
      srk_set: oem # [Required], Super Root Key (SRK) set, Defines which set is used to authenticate the container., Possible options:['none', 'oem', 'nxp']
      used_srk_id: 0 # [Conditionally required], Used SRK, Which key from SRK set is being used.
      srk_revoke_mask: 0 # [Optional], SRK revoke mask, Bitmask to indicate which SRKs to revoke. Bit set to 1 means revoke key. Bit 0 = revoke SRK_0, bit 1 = revoke SRK_1 etc.
      fuse_version: 0 # [Required], Fuse version, The value must be equal or greater than the version stored in fuses to allow loading this container.
      sw_version: 0 # [Required], Software version, Number used by Privileged Host Boot Companion (PHBC) to select between multiple images with same Fuse version field.
      signing_key:
        inputs/keys/ecc256/srk0_ecc256.pem # [Conditionally required], AHAB container signing key, Private key used for sign the container header. Header can be signed by SRK or by image key that was signed by SRK. If an image key is used, it must be the same algorithm and key size as the SRK. In both cases, the referenced SRK must not have been revoked.
        # ----------------------------------------------------------------------------------------------------
        #               == Configuration of AHAB Container images (array of multiple images) ==
        # ----------------------------------------------------------------------------------------------------
      images: # [Required], Image array, Array of image entries.
        - image_path: inputs/mx93_mboot_0x304A0000.bin # [Required], Image path, Path to image binary (absolute/relative).
          image_offset: "0x10000" # [Required], Image offset in AHAB container, Relative address for start of AHAB image (can contain multiple AHAB containers). In case of XiP type of AHAB image, the load_address and entry_point must correspond to this values. Example of setting of load_address - AHAB_IMAGE_ADDRESS+IMAGE_OFFSET=LOAD_ADDRESS
          load_address: "0x304A0000" # [Required], Image destination address, Address the image is written to in memory (absolute address in system memory).
          entry_point: "0x304A0000" # [Required], Image entry point, Image entry point (absolute address). Valid only for executable image types.
          image_type: executable # [Required], Image type, Kind of image., Possible options:['executable', 'data', 'dcd_image', 'seco', 'provisioning_image', 'provisioning_data']
          core_id: cortex-m33 # [Required], Core ID, Defines the core the image is dedicated for., Possible options:['cortex-m33', 'cortex-m7']
          is_encrypted: false # [Required], Image encryption, Determines, whether image is encrypted or not.
          boot_flags: 0 # [Optional], Boot flags, Boot flags controlling SCFW boot.
          meta_data_start_cpu_id: 0 # [Optional], Start CPU ID, Resource ID of CPU to be started
          meta_data_mu_cpu_id: 0 # [Optional], CPU memory unit start ID, Resource ID of the MU associated with the CPU
          meta_data_start_partition_id: 0 # [Optional], Start partition ID, Partition ID of the partition to start
          hash_type: sha384 # [Optional], Images HASH type, HASH type of image. All images in the container must have the same HASH type., Possible options:['sha256', 'sha384', 'sha512']
        # ----------------------------------------------------------------------------------------------------
        #                                == Configuration of AHAB SRK table ==
        # ----------------------------------------------------------------------------------------------------
      srk_table: # [Conditionally required], SRK Table, SRK (Super Root key) table definition.
        srk_array: # [Required], Super Root Key (SRK) table, Table containing the used SRK records. All SRKs must be of the same type. Supported signing algorithms are: RSASSA-PSS or ECDSA. Supported hash algorithms: sha256, sha384, sha512. Supported key sizes/curves: prime256v1, sec384r1, sec512r1, rsa2048, rsa4096. Certificate may be of Certificate Authority.
          - inputs/keys/ecc256/srk0_ecc256.pub # SRK key, Path to SRK Key file.
          - inputs/keys/ecc256/srk1_ecc256.pub # SRK key, Path to SRK Key file.
          - inputs/keys/ecc256/srk2_ecc256.pub # SRK key, Path to SRK Key file.
          - inputs/keys/ecc256/srk3_ecc256.pub # SRK key, Path to SRK Key file.


In [4]:
%alias execute echo %l && %l
%alias_magic ! execute

WORKSPACE = "workspace/" # change this to path to your workspace
VERBOSITY = "-v" # verbosity of commands, might be -v or -vv for debug or blank for no additional info
AHAB_CONFIGURATION = "ahab_config_flashloader_signed256.yaml"
AHAB_IMAGE = "outputs/flashloader_signed.bin"

# export AHAB image with flashloader
%! nxpimage $VERBOSITY ahab export -c $AHAB_CONFIGURATION 



Created `%!` as an alias for `%execute`.
nxpimage -v ahab export -c ahab_config_flashloader_signed256.yaml 
Success. (AHAB: C:/Users/nxf46245/spsdk/examples/jupyter_examples/imx93/outputs/flashloader_signed.bin created.)
Generated SRK hash files (c:\Users\nxf46245\spsdk\examples\jupyter_examples\imx93\flashloader_signed_oem1_srk_hash*.*).


INFO:spsdk.apps.nxpimage:Created AHAB Image:
Name:      AHAB Image
Starts:    0x0
Ends:      0x147a7
Size:      Size: 83.9 kB; 83,880 B
Alignment: 1 B
AHAB Image for mx93_a0

INFO:spsdk.apps.nxpimage:Created AHAB Image memory map:

+==0x0000_0000= AHAB Image =======================+
|             Size: 83.9 kB; 83,880 B             |
|             AHAB Image for mx93_a0              |
|+==0x0000_0000= AHAB Container 0 ===============+|
||                  Size: 544 B                  ||
||        AHAB Container for nxp_SWver:0         ||
|+==0x0000_021f==================================+|
|                   Gap: 480 B                    |
|+==0x0000_0400= AHAB Container 1 ===============+|
||                  Size: 544 B                  ||
||        AHAB Container for oem_SWver:0         ||
|+==0x0000_061f==================================+|
|                   Gap: 6.6 kB                   |
|+==0x0000_2000= Container 0 AHAB Data Image 0 ==+|
||            Size: 58.1 kB; 58,144 B   

## Image download
We use SPSDK tool *sdpshost* to load the image. So at first, we need to set up board boot configuration to Cortex-M serial downloader mode (1011) and load the image to OCRAM. Application should start immediately after loading the image.

In [36]:
# use SDPSHOST tool to download the image
! sdpshost $VERBOSITY -u mx93 -n MX93 write-file outputs/flashloader_signed.bin
 


!!! THIS IS AN EXPERIMENTAL UTILITY! USE WITH CAUTION !!!


!!! THIS IS AN EXPERIMENTAL UTILITY! USE WITH CAUTION !!!

INFO:spsdk.sdp.sdps:Connect: OO Blank 93 (0x1FC9, 0x014E)
INFO:spsdk.sdp.sdps:TX-CMD: WriteFile(length=83888)


INFO:libusbsio:Loading SIO library: C:\Users\nxf46245\spsdk\venv\Lib\site-packages\libusbsio\bin\x64\libusbsio.dll
INFO:libusbsio:HID enumeration[2652857619696]: initialized
INFO:libusbsio:HID enumeration[2652857619696]: finished, total 14 devices
INFO:libusbsio.hidapi.dev:Opening HID device at path: 'b'\\\\?\\hid#vid_1fc9&pid_014e#6&6466ab0&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}''
INFO:libusbsio.hidapi.dev:HID device 2652814117008 is now open
INFO:libusbsio.hidapi.dev:HID device 2652814117008 closed


## Nxpdevscan
The flashloader application should start immediately after loading to OCRAM.

To determine the port where the flashloader communicates we can use the *nxpdevscan* app.

In [37]:
# Check which port is used by flashloader

%! nxpdevscan -p 

nxpdevscan -p 
-------- Connected NXP UART Devices --------

Port: COM41
Type: mboot device



# BLHOST

To verify the communication with blhost we can try the blhost get-property command.

In [38]:
# modify the UART port according to the nxpdevscan
CONNECTION = "-p com41"
# get-property 1 returns version of the flashloader
%! blhost $CONNECTION get-property 1

blhost -p com41 get-property 1 
Response status = 0 (0x0) Success.
Response word 1 = 1258424320 (0x4b020800)
Current Version = K2.8.0


In [39]:
# get property 18 returns UUID of the device
%! blhost $CONNECTION get-property 18

blhost -p com41 get-property 18 
Response status = 0 (0x0) Success.
Response word 1 = 2442643059 (0x9197c673)
Response word 2 = 2573725851 (0x9967f09b)
Response word 3 = 1195651327 (0x474430ff)
Response word 4 = 1979819762 (0x7601a6f2)
Unique Device Identification = 73 C6 97 91 9B F0 67 99 FF 30 44 47 F2 A6 01 76


In [43]:
# get property 7 returns available commands
%! blhost $CONNECTION get-property 7

blhost -p com41 get-property 7 
Response status = 0 (0x0) Success.
Response word 1 = 25227228 (0x180efdc)
Available Commands = ['ReadMemory', 'WriteMemory', 'FillMemory', 'GetProperty', 'ReceiveSBFile', 'Execute', 'Call', 'Reset', 'SetProperty', 'FlashProgramOnce', 'FlashReadOnce', 'FlashReadResource', 'UpdateLifeCycle', 'EleMessage']


In [48]:
# Lets's read some data from OCRAM with hexdump output
%! blhost $CONNECTION read-memory 0x304f0000 100 -h

blhost -p com41 read-memory 0x304f0000 100 -h 
Reading memory
00000000: 1C DB F7 7A 59 B9 0C 96  AC 9F F7 E1 ED 51 EE DD  ...zY........Q..
00000010: ED 22 88 C9 2F B2 EC 0B  7D 15 56 33 FC 22 2E 72  ."../...}.V3.".r
00000020: 33 56 22 73 2A 2F 1F 00  B7 29 E6 F8 9D 0B C4 E1  3V"s*/...)......
00000030: C2 E9 0F 77 33 CC 31 D4  48 14 01 99 D9 C6 60 6A  ...w3.1.H.....`j
00000040: FB 17 39 E3 4A 8D 4C 01  4D 68 9D 95 E5 EE 6D 43  ..9.J.L.Mh....mC
00000050: 60 23 56 68 F0 83 D1 61  03 EE 8E E5 79 88 12 04  `#Vh...a....y...
00000060: E8 74 53 84                                       .tS.
Response status = 0 (0x0) Success.
Response word 1 = 100 (0x64)
Read 100 of 100 bytes.


## NXPELE

NXPELE is a tool that can communicate with the EdgeLock enclave. It might be used for example for writing and reading the fuses and keyblob generation.

In [12]:
# get information about the ELE
%! nxpele -f mx93 $CONNECTION get-info

nxpele -f mx93 -p com41 get-info 
ELE get info ends successfully:
Command:          0xda
Version:          1
Length:           92
SoC ID:           9300
SoC version:      A000
Life Cycle:       OEM_OPEN - 0x0010
SSSM state:       4
UUID:             f2a60176ff3044479bf0679973c69791
SHA256 ROM PATCH: 0563db6df6388ec7f5b2bba52110bebdfa0302b6e3a39493e03c7c9be7ab8686
SHA256 FW:        4d352fbbd4ecaac4359847e98f7851ac3f05e105005363ed5ff244b2a64176e9



In [25]:
# read fuse with index 128 OEM_SRKH[255:224]
%! nxpele -f mx93 $CONNECTION read-common-fuse -i 128

nxpele -f mx93 -p com41 read-common-fuse -i 128 
Read common fuse ends successfully.
Fuse ID_128: 0x00000000



In [40]:
# read shadow fuse with index 320 MAC4_ADDR_31_0[31:0] (Reserved)
%! nxpele -f mx93 $CONNECTION read-shadow-fuse -i 320

nxpele -f mx93 -p com41 read-shadow-fuse -i 320 
Read shadow fuse ends successfully.
Fuse ID_320: 0x00000000



In [41]:
# write shadow fuse with index index 320 MAC4_ADDR_31_0[31:0] (Reserved)
%! nxpele -f mx93 $CONNECTION write-shadow-fuse -i 320 -d 0x12345678

nxpele -f mx93 -p com41 write-shadow-fuse -i 320 -d 0x12345678 
ELE write shadow fuse ends successfully.


In [42]:
# read shadow fuse with index index 320 MAC4_ADDR_31_0[31:0] (Reserved)
%! nxpele -f mx93 $CONNECTION read-shadow-fuse -i 320

nxpele -f mx93 -p com41 read-shadow-fuse -i 320 
Read shadow fuse ends successfully.
Fuse ID_320: 0x12345678

